In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path

import nrrd
import numpy as np
import SimpleITK as sitk
import torch
from tqdm import tqdm

from utils import process_img_array, load_process_csv
from matplotlib import pyplot as plt
from experiments.utils import generate_seg
from experiments.model_utils import load_medsam

device = 'cuda' if torch.cuda.is_available() else 'cpu'; print(device)
medsam = load_medsam("../work_dir/MedSAM/medsam_vit_b.pth", device)

In [87]:
data_folder = Path("/media/liushifeng/KINGSTON/ULS Jan 2025/ULS23/novel_data/ULS23_DeepLesion3D")
output_folder = Path("/home/liushifeng/Documents/GitHub/MedSAM/experiments/outputs/extend_3d/ULS_windowed_10Feb2025/")  # ULS_windowed_10Feb2025  ULS_full_10Feb2025

In [ ]:
with open(data_folder / "train.txt") as f:
    train_names = [x.rstrip() for x in f.readlines()]
with open(data_folder / "val.txt") as f:
    val_names = [x.rstrip() for x in f.readlines()]

In [ ]:
for filename in tqdm(val_names[126:]):
    print(f"Segmenting {filename}")
    img_path = data_folder / "images" / filename
    seg_path = data_folder / "labels" / filename
    break

In [ ]:
csv_folder = Path("/home/liushifeng/Desktop/DeepLesion Dataset/")
df = load_process_csv(csv_folder / "DL_info.csv")
LESION_TYPE_WINDOWS = {-1: None, 1: 'bone', 2: 'abdomen', 3: 'abdomen', 4: 'abdomen',
     5: 'lung', 6: 'abdomen', 7: 'abdomen', 8: 'abdomen'}
df['ct_window'] = df['Coarse_lesion_type'].map(LESION_TYPE_WINDOWS)

In [ ]:
ct = sitk.ReadImage(img_path)
seg = sitk.ReadImage(seg_path)
ct_array = sitk.GetArrayFromImage(ct)
seg_array = sitk.GetArrayFromImage(seg)

In [ ]:
vol_seg = generate_seg(img_path, seg_path, medsam)

In [ ]:
np.where(vol_seg.sum(axis=(1,2)) > 0)

In [ ]:
output_folder

In [89]:
from collections import Counter
Counter(window_types)

Counter({'abdomen': 127, 'bone': 26, 'lung': 20})

In [95]:
output_folder

PosixPath('/home/liushifeng/Documents/GitHub/MedSAM/experiments/outputs/extend_3d/ULS_windowed_10Feb2025')

In [ ]:
window_types = []
for filename in tqdm(train_names):

    orig_name = filename.split("_lesion_")[0]
    window = df[df['File_name'].str.contains(orig_name)].iloc[:1]['ct_window'].item()

    if window is None:
        continue
    window_types.append(window)

    # window = None
    img_path = data_folder / "images" / filename
    seg_path = data_folder / "labels" / filename

    print(f"Segmenting {filename} in {window=}")

    vol_seg = generate_seg(
        img_path, seg_path, medsam,
        None, # "ULS_full_10Feb2025_viz",
        window,
        plot=True
    )
    # save nrrd 3d seg for eval
    nrrd.write(f"{output_folder}/{filename}_pred.nrrd", vol_seg)

In [ ]:
l, a = process_img_array(ct_array[60])